In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfTransformer
from keras.preprocessing.text import text_to_word_sequence
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

## 1. Prepare data

In [2]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/ML/preprocessing')
import Preprocessor

In [3]:
X_train, y_train, X_test, y_test = Preprocessor.file_read_vectorizer('/content/drive/MyDrive/ML/data/TrainingSet.csv', '/content/drive/MyDrive/ML/data/TestSet.csv')

## 2. Model training

### 2.1. Individual model performances

In [4]:
# import model libraries
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [5]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy')
lr = LogisticRegression()
svm = LinearSVC(C=5.0)
nb = MultinomialNB()

for clf in (rf, lr, svm, nb):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.8615
LogisticRegression 0.88845
LinearSVC 0.90585
MultinomialNB 0.8848


### 2.2. Voting classifier

In [6]:
rf = RandomForestClassifier(n_estimators=300, criterion='entropy')
lr = LogisticRegression(C=5)
svm = LinearSVC(C=150, max_iter=10000)
nb = MultinomialNB()

voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('nb', nb)],
    voting='hard')
voting_clf.fit(X_train, y_train)
print(voting_clf.__class__.__name__, accuracy_score(y_test, voting_clf.predict(X_test)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


VotingClassifier 0.89495
